In [1]:
import requests
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet_ic')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wxp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
nltk.__version__

'3.4.4'

In [163]:
url = "http://sitesearch.asahi.com/ajwsitesearch/index.php?Keywords=Hong%20Kong&start=0"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [122]:
soup.findAll('span',{'class':'SearchResult_Headline'})

[<span class="SearchResult_Headline"><em>In leaderless <strong>Hong</strong> <strong>Kong</strong> movement, Joshua Wong just 1 voice</em><span class="EnDate">September 17, 2019</span></span>,
 <span class="SearchResult_Headline"><em>Amid protests, Moody’s downgrades <strong>Hong</strong> <strong>Kong</strong> to ‘negative’</em><span class="EnDate">September 17, 2019</span></span>,
 <span class="SearchResult_Headline"><em>Taiwan: China meddling with elections after Solomon Islands cuts ties</em><span class="EnDate">September 17, 2019</span></span>,
 <span class="SearchResult_Headline"><em>Hospital staff rally for protest campaign after new violence</em><span class="EnDate">September 17, 2019</span></span>,
 <span class="SearchResult_Headline"><em><strong>Hong</strong> <strong>Kong</strong> reopens after weekend of clashes, protests</em><span class="EnDate">September 16, 2019</span></span>,
 <span class="SearchResult_Headline"><em>Oil prices leap as attack on Saudi facility disrupts out

In [167]:
a = soup.findAll('ul',{'id':'SiteSearchResult'})[0]
a.findAll('a')[9]['href']

'http://www.asahi.com/ajw/articles/AJ201909140014.html'

In [168]:
url = 'http://www.asahi.com/ajw/articles/AJ201909170026.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [158]:
soup.findAll('p',{'class':'EnArticleName'})[0].contents[0]

'THE ASSOCIATED PRESS'

In [159]:
soup.findAll('p',{'class':'EnLastUpdated'})[0].contents[0]

'September 17, 2019 at 12:35 JST'

In [160]:
soup.findAll('p',{'class':'EnLastUpdated'})[0].contents[0].split()

['September', '17,', '2019', 'at', '12:35', 'JST']

In [172]:
len(soup.findAll('div',{'class':'ArticleText'})[0].findAll('p'))

24

In [169]:
soup.findAll('div',{'class':'ArticleText'})[0].findAll('p')[1].contents[0]

'The 22-year-old activist, who rose to fame as a leader of democracy protests five years ago, speaks to a U.S. congressional committee on Tuesday, following visits to Germany and Taiwan to drum up support for the movement.'

In [171]:
soup.findAll('div',{'class':'ArticleText'})[0].findAll('p')[0].contents[0]

"HONG KONG--Overseas, Joshua Wong has emerged as a prominent face of Hong Kong's months-long protests for full democracy. At home, he is just another protester."

In [153]:
a = soup.findAll('div',{'class':'ArticleText'})[0].findAll('p')[1].contents[0]

In [161]:
"http://sitesearch.asahi.com/ajwsitesearch/index.php?Keywords=Hong%20Kong&start="+str(10)

'http://sitesearch.asahi.com/ajwsitesearch/index.php?Keywords=Hong%20Kong&start=10'

In [238]:
import json
Japan = {}
for i in range(40):
    # get into the search page
    idx = 10*i
    url = "http://sitesearch.asahi.com/ajwsitesearch/index.php?Keywords=Hong%20Kong&start="+str(idx)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a = soup.findAll('ul',{'id':'SiteSearchResult'})[0]
    # get the info for the article link, 10 articles for each page
    for j in range(10):
        uu = a.findAll('a')[j]['href']
        response = requests.get(uu)
        ss = BeautifulSoup(response.text, 'html.parser')
        try:
            author = ss.findAll('p',{'class':'EnArticleName'})[0].contents[0]
        except:
            author = 'editor'
        title = ss.findAll('h1')[0].contents[0]
        try:
            time = ss.findAll('p',{'class':'EnLastUpdated'})[0].contents[0].split()
        except:
            time = 'unknown'
        body = ss.findAll('div',{'class':'ArticleText'})[0].findAll('p')
        # remove special character in the body
        article = ''
        for k in range(len(body)):
            if len(body[k].contents)>0:
                if isinstance(body[k].contents[0], str):
                    article = article + body[k].contents[0]
            
        # save to json file
        key = title + ',' + author + ',' + time[0] + ',' + time[1] + time[2]
        Japan[key] = article

In [243]:
# save as json
with open('Japan.json', 'w') as outfile:
    json.dump(Japan, outfile)

In [3]:
# load json
import json
with open('Japan.json', 'r') as outfile:
        Japan = json.load(outfile)

In [4]:
list(Japan.keys())

['In leaderless Hong Kong movement, Joshua Wong just 1 voice,THE ASSOCIATED PRESS,September,17,2019',
 'Amid protests, Moodyâ\x80\x99s downgrades Hong Kong to â\x80\x98negativeâ\x80\x99,REUTERS,September,17,2019',
 'Taiwan: China meddling with elections after Solomon Islands cuts ties,REUTERS,September,17,2019',
 'Hospital staff rally for protest campaign after new violence,THE ASSOCIATED PRESS,September,17,2019',
 'Hong Kong reopens after weekend of clashes, protests,REUTERS,September,16,2019',
 'Oil prices leap as attack on Saudi facility disrupts output,THE ASSOCIATED PRESS,September,16,2019',
 'Violence flares after Hong Kong protesters defy police ban,THE ASSOCIATED PRESS,September,16,2019',
 'Protesters defy ban to march in downtown Hong Kong,THE ASSOCIATED PRESS,September,15,2019',
 'Hong Kong police break up clashes between rival protesters,REUTERS,September,14,2019',
 'Lantern-waving Hong Kong protesters take to hills,REUTERS,September,14,2019',
 'London Stock Exchange rejects

In [1]:
# filter the Japan, remove the unrelated document
import nltk
import pandas as pd
import numpy as np

In [7]:
document = ''
for i in range(len(list(Japan.values()))):
    document += list(Japan.values())[i]

In [9]:
doc_tokens = nltk.word_tokenize(document)
text1 = nltk.Text(doc_tokens)

In [12]:
# number of unique tokens
print(len(set(text1)))
# After lemmatizing, the number of unique words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(w,'v') for w in text1]
print(len(set(lemmatized)))    

20828
17943


In [13]:
# 20 most frequent unique tokens
import collections
from operator import itemgetter  
dist = nltk.FreqDist(text1)
sorted_dist = collections.OrderedDict(sorted(dist.items(), key = itemgetter(1), reverse = True))
print(list(sorted_dist.items())[:20])

[('the', 12302), (',', 11807), ('to', 7337), ('of', 6238), ('and', 5990), ('in', 5655), ('a', 5212), ('.', 4495), ('``', 3472), ("'s", 3012), ('on', 2490), ('Kong', 2410), ('Hong', 2403), ('that', 2394), ('for', 2307), ('said', 2127), ("''", 2042), ('China', 1650), ('with', 1530), ('is', 1522)]


In [15]:
chosed_tokens = [x for x in dist.keys() if len(x) > 5 and dist[x] > 150]
chosed_tokens = sorted(chosed_tokens)
print(chosed_tokens)    

['Beijing', 'British', 'Carrie', 'Chinese', 'Communist', 'Friday', 'Huawei', 'Monday', 'President', 'States', 'Sunday', 'Taiwan', 'Tuesday', 'United', 'Wednesday', 'against', 'because', 'before', 'between', 'billion', 'business', 'called', 'companies', 'company', 'country', 'demands', 'demonstrations', 'demonstrators', 'extradition', 'financial', 'foreign', 'former', 'freedoms', 'global', 'government', 'including', 'international', 'leader', 'mainland', 'million', 'months', 'official', 'people', 'percent', 'police', 'political', 'pro-democracy', 'protest', 'protesters', 'protests', 'public', 'recent', 'rights', 'support', 'systems', 'territory', 'through', 'violence', 'violent']


In [48]:
key

'Thousands take to Hong Kong streets to protest new extradition laws,REUTERS,April,28,2019'

In [47]:
key = list(Japan.keys())[327]
value = Japan[key]
value

'HONG KONG--Thousands of people marched on Hong Kong\'s parliament on Sunday to demand the scrapping of proposed extradition rules that would allow people to be sent to mainland China for trial--a move which some fear puts the city\'s core freedoms at risk.Opponents of the proposal fear further erosion of rights and legal protections in the free-wheeling financial hub--freedoms which were guaranteed under the city\'s handover from British colonial rule to Chinese sovereignty in 1997.Early estimates suggested several thousand people had joined the march along Hong Kong Island from Causeway Bay to the council in the Admiralty business district.Veteran Hong Kong activist and former legislator Leung Kwok-hung said the government\'s move risked removing Hong Kongers\' "freedom from fear.""Hong Kong people and visitors passing by Hong Kong will lose their right not to be extradited into mainland China," he said. "They would need to face an unjust legal system on the mainland."The proposed ch

In [49]:
Japan_filter = {}
for i in range(327):
    key = list(Japan.keys())[i]
    value = Japan[key]
    if 'protest' in value or 'Protest' in value or 'protests' in value or 'Protests' in value:
        Japan_filter[key] = value

In [50]:
# save as json
with open('Japan_filter.json', 'w') as outfile:
    json.dump(Japan_filter, outfile)

In [1]:
# load json
import json
with open('Japan_filter.json', 'r') as outfile:
        Japan = json.load(outfile)

In [5]:
len(list(Japan.keys()))

235

In [22]:
# path similarity
deer = wn.synset('deer.n.01')
elk = wn.synset('elk.n.01')
horse = wn.synset('horse.n.01')
print(deer.path_similarity(elk))
print(deer.path_similarity(horse))

0.5
0.14285714285714285


In [29]:
# Lin similarity
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
print(deer.lin_similarity(elk,brown_ic))
print(deer.lin_similarity(horse,brown_ic))

0.8623778273893673
0.7726998936065773


In [31]:
# Distributional similarity: context
# Words before, after, within a small window
# Pointwise Mutual Information PMI(w,c) = log[P(w,c)/P(w)P(c)]
import nltk 
from nltk.collocations import *

#bigram_measures = nltk.collocations.BigramAssocMeasures()
#finder = BigramCollocationFinder.from_words(text)
#finder.nbest(bigram_measures.pmi,10)
#finder.apply_freq_filter(10)

In [2]:
# textrank with gensim
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords

In [4]:
list(Japan.values())[0]

'HONG KONG--Overseas, Joshua Wong has emerged as a prominent face of Hong Kong\'s months-long protests for full democracy. At home, he is just another protester.The 22-year-old activist, who rose to fame as a leader of democracy protests five years ago, speaks to a U.S. congressional committee on Tuesday, following visits to Germany and Taiwan to drum up support for the movement.While not diminishing the importance of that role, other protesters say Wong does not speak for what is purposefully a leaderless movement. He has received media coverage, both because he is well-known and the movement is largely faceless, since many protesters speak only anonymously and wear masks to try to avoid arrest."Not that nobody cares about what he says, but it\'s just that Joshua Wong alone cannot represent the whole of Hong Kong," said Sean Au, a 17-year-old student. "He is just a participant, no longer a leader."His activities have nonetheless made him a target of the Chinese government, which has u

In [5]:
print(summarize(list(Japan.values())[0],word_count=300))

He remains secretary-general of the organization, which now describes itself on Twitter as a movement-oriented youth activist group.Wong, who has been arrested and jailed repeatedly, was released in June, days after more than a million people took to the streets to protest an extradition bill which is widely regarded as an example of China chipping away at the city's autonomy.He was rearrested last month along with several others and charged with organizing an illegal rally outside a police station in June.While Wong has inspired many, neither he nor other well-known individuals or groups have a central role in this summer's mass movement, said Bonnie Leung of the Civil Human Rights Front, which has organized several massive marches this summer.Leung acknowledged that Wong's reputation as the poster child of the 2014 protest gives him stature as a protest representative on the global stage, along with others such as Hong Kong pop star Denise Ho."Globally we need actions that will make 

In [14]:
print(summarize(list(Japan.values())[0],word_count=100))

"Joshua Wong is a well-known face across the world and he can help in this."Wong and Ho are among five people due to speak at a hearing of the U.S. Congressional-Executive Commission on China titled "Hong Kong's Summer of Discontent and U.S. Policy Responses."They hope to rally support for the Hong Kong Human Rights and Democracy Act, a bill proposing economic sanctions and penalties on Chinese and Hong Kong officials found to have suppressed democracy in the city.Mousse Chan, a 45-year-old teacher, said she supports Wong in his global tour, though she echoed the view that he is not their leader."We believe that every Hong Kong citizen has a responsibility to raise awareness about our cause and he is just one of them," she said.Wong has accused the government of trying to frame prominent activists such as himself as a warning to other protesters, but he said it was doomed to fail as the current unrest has no centralized figureheads.Born in 1996 to a middle-class Christian family, Wong 

In [23]:
#keywords(list(Japan.values())[0]).split('\n')

In [25]:
# similarity calculation
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

text = list(Japan.values())[0]
stops = set(stopwords.words("english"))
words = [w for w in text.lower().split() if not w in stops]
final_text = " ".join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wxp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [27]:
# remove special charaters
review_text = re.sub(r"[^A-Za-z0-9(),!.?\'`]", " ", final_text )
review_text = re.sub(r"\'s", " 's ", final_text )
review_text = re.sub(r"\'ve", " 've ", final_text )
review_text = re.sub(r"n\'t", " 't ", final_text )
review_text = re.sub(r"\'re", " 're ", final_text )
review_text = re.sub(r"\'d", " 'd ", final_text )
review_text = re.sub(r"\'ll", " 'll ", final_text )
review_text = re.sub(r",", " ", final_text )
review_text = re.sub(r"\.", " ", final_text )
review_text = re.sub(r"!", " ", final_text )
review_text = re.sub(r"\(", " ( ", final_text )
review_text = re.sub(r"\)", " ) ", final_text )
review_text = re.sub(r"\?", " ", final_text )
review_text = re.sub(r"\s{2,}", " ", final_text )

In [48]:
final_text

'hong kong--overseas, joshua wong emerged prominent face hong kong\'s months-long protests full democracy. home, another protester.the 22-year-old activist, rose fame leader democracy protests five years ago, speaks u.s. congressional committee tuesday, following visits germany taiwan drum support movement.while diminishing importance role, protesters say wong speak purposefully leaderless movement. received media coverage, well-known movement largely faceless, since many protesters speak anonymously wear masks try avoid arrest."not nobody cares says, joshua wong alone cannot represent whole hong kong," said sean au, 17-year-old student. "he participant, longer leader."his activities nonetheless made target chinese government, used accuse foreign powers colluding anti-china separatists foment unrest.wong\'s activism started age 13 joined protests proposed high-speed rail link hong kong mainland china. link opened last year many delays.the bespectacled teen set student activist group, s

In [49]:
final_text.split('.')[0]

"hong kong--overseas, joshua wong emerged prominent face hong kong's months-long protests full democracy"

In [73]:
labeled_questions=[]
labeled_questions.append(TaggedDocument(final_text.split(),range(len(final_text.split()))))

In [74]:
model = Doc2Vec(dm = 1, min_count=1, window=10, size=150, sample=1e-4, negative=10)
model.build_vocab(labeled_questions)

In [75]:
for epoch in range(20):
    model.train(labeled_questions,epochs=model.iter,total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

C:\Users\wxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.
Epoch #4 is complete.
Epoch #5 is complete.
Epoch #6 is complete.
Epoch #7 is complete.
Epoch #8 is complete.
Epoch #9 is complete.
Epoch #10 is complete.
Epoch #11 is complete.
Epoch #12 is complete.
Epoch #13 is complete.
Epoch #14 is complete.
Epoch #15 is complete.
Epoch #16 is complete.
Epoch #17 is complete.
Epoch #18 is complete.
Epoch #19 is complete.
Epoch #20 is complete.


In [59]:
model.most_similar('hong kong--overseas, joshua wong emerged prominent face hong kong\'s months-long protests full democracy')

C:\Users\wxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('the 22-year-old activist, rose fame leader democracy protests five years ago, speaks u',
  0.12463772296905518),
 (' home, another protester', 0.122573122382164),
 ('"wong ho among five people due speak hearing u', 0.11188612878322601),
 (" protesters occupied major thoroughfares semiautonomous chinese territory 79 days ultimately unsuccessful push direct elections city's leaders",
  0.1094951182603836),
 ('wong co-founded political party, demosisto, 2016, members disqualified serving later even running office advocated self-determination',
  0.09625684469938278),
 ("wong's activism started age 13 joined protests proposed high-speed rail link hong kong mainland china",
  0.08907414227724075),
 ('he later compared hong kong east germany pro-democracy protests led fall berlin wall',
  0.08744935691356659),
 (' policy responses', 0.08557818830013275),
 ('"i see reason us give time world stand hong kong," said',
  0.08464326709508896),
 ('while diminishing importance role, protesters sa

In [76]:
questions_split = final_text.split()

In [77]:
questions_split[2]

'joshua'

In [79]:
model.most_similar(questions_split[2])

C:\Users\wxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('kong', 0.9944936633110046),
 ('protest', 0.9938367605209351),
 ('clear', 0.9934786558151245),
 ('democracy', 0.9932013750076294),
 ('schools.', 0.9931670427322388),
 ('opened', 0.9930024147033691),
 ('whole', 0.9929667115211487),
 ('government,', 0.9929406046867371),
 ('speak', 0.9929075837135315),
 ('along', 0.9928964376449585)]

In [81]:
model.infer_vector('the 22-year-old activist, rose fame leader democracy protests five years ago, speaks u'.split())

array([-0.02381895, -0.07758579, -0.01368993, -0.02099467, -0.02295201,
        0.0049884 ,  0.03978407,  0.05120877, -0.00455867,  0.03973684,
        0.02516356,  0.01598408,  0.01602839,  0.02399345,  0.01917564,
       -0.00877979, -0.02145232,  0.03905983, -0.05370756,  0.03195766,
       -0.03669301, -0.032611  , -0.02022472,  0.01322896, -0.02139915,
        0.00080295,  0.02365625, -0.048961  ,  0.01010074, -0.0088423 ,
        0.03939902,  0.03341983, -0.04525749, -0.03837388, -0.01021375,
        0.02937328,  0.00797123,  0.02947779,  0.01149655,  0.02044325,
        0.01130801,  0.02682213,  0.00159786, -0.0104731 ,  0.01330418,
        0.03473864, -0.00685804,  0.01237351,  0.02863524,  0.00429843,
        0.00048975, -0.03173163,  0.05038798,  0.01045189,  0.04773301,
        0.00185695, -0.03136143,  0.02745213, -0.01501341,  0.00090314,
       -0.01514976,  0.01275675, -0.00581999, -0.01831162, -0.00993098,
        0.00386462,  0.01478254, -0.01900079,  0.02507095, -0.01